In [ ]:
import os
import sys  
import json
import torch 
import argparse
import numpy as np
from PIL import Image  
from tqdm import tqdm
from utils import model_gen
from transformers import AutoModelForCausalLM, AutoTokenizer  

ckpt_path = 'internlm/internlm-xcomposer2-vl-7b'
tokenizer = AutoTokenizer.from_pretrained(ckpt_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(ckpt_path, device_map="cuda", trust_remote_code=True).eval().cuda().half()
model.tokenizer = tokenizer

In [ ]:
samples = json.load(open('data/hallu/HallusionBench.json'))

results = []
for q in tqdm(samples):   
    if q['filename'] == None:
        continue  ### only evaluate the image part
        
    try: #### correct some file name problem
        im_path = 'data/hallu/hallusion_bench/' + q['filename']
        _ = Image.open(im_path)
    except:
        if q['filename'].find('png') != -1:
            im_path = 'data/hallu/hallusion_bench/' + q['filename'].replace('png', 'PNG')
        else:
            im_path = 'data/hallu/hallusion_bench/' + q['filename'].replace('PNG', 'png')
             
    text = '[UNUSED_TOKEN_146]user\n{}[UNUSED_TOKEN_145]\n[UNUSED_TOKEN_146]assistant\n'.format(q['question'])
    with torch.cuda.amp.autocast(): 
        response = model_gen(model, text, im_path, padding=True)
    ans = '1' if response.lower().find('yes')!=-1 else '0'
    results.append(ans == q['gt_answer'])
    q.append(ans)
    
print ("aAcc:", np.mean(results))


In [ ]:
qlist = {}
for r in samples:
    key = "_".join([r["category"], r["subcategory"], str(r["set_id"]), str(r["question_id"])])
    try:
        qlist[key].append(r['answer'] == r['gt_answer'])
    except:
        qlist[key] = [r['answer'] == r['gt_answer']]
out = []
for q, v in qlist.items(): 
    out.append(min(v))
print ("qAcc:", np.mean(out))

In [ ]:
qlist = {}
for r in samples:
    key = "_".join([r["category"], r["subcategory"], str(r["set_id"]), str(r["figure_id"])])
    try:
        qlist[key].append(r['answer'] == r['gt_answer'])
    except:
        qlist[key] = [r['answer'] == r['gt_answer']]
out = []
for q, v in qlist.items(): 
    out.append(min(v))
print ("fAcc:", np.mean(out))